In [1]:
import core
import pandas as pd
from datetime import datetime
import ast

In [2]:
current_date = "2025-07-23"
datetime_obj = pd.to_datetime(current_date)

In [ ]:
core.MonteCarloSim(match_id=4640)

Simulating Santos vs Internacional


Simulations (1 worker): 100%|██████████| 100/100 [00:00<00:00, 249.95it/s]
